In [2]:
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

n_instances = 100
subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:n_instances]]
len(subj_docs), len(obj_docs)

(100, 100)

In [8]:
import pandas as pd

df = pd.DataFrame()
df = pd.read_csv('./dota2-predictor/dota2_data/dota2_chat2.csv')
print(df.head())
print(df.tail())

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.tokenize.regexp import RegexpTokenizer 
import nltk
import numpy as np
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
tokenizer = RegexpTokenizer("[\w']+") 

docs = df['key']
# print(docs[10031])

a_docs  = np.array(docs)

count = CountVectorizer()
ts = []
# 1120648
print(len(a_docs))
length = len(a_docs)

for i in range(0, 29999):
    to = tokenizer.tokenize(a_docs[i])
    if len(to) > 0:
        for j in to:
            ts.append(stemmer.stem(j.lower()))


    
bag = count.fit_transform(ts)

# # print(count.vocabulary_)
print(bag.toarray())

en = nltk.Text(ts)

en.vocab().items()

a = nltk.FreqDist(en)


fdist = nltk.FreqDist(en.vocab())

for word, frequency in fdist.most_common(400):
    print(u'{};{}'.format(word, frequency))

     match_id  time  type                     unit  key  slot  player_slot
0  2304329360  2728  chat                   Darren   gg     2            2
1  2304329360  2733  chat                haTaTiTla   gg     1            1
2  2304342708   -45  chat  /.=^-^=./ YO YOY YOY YO   ^^     3            3
3  2304342708    -3  chat               DarkRaider  thx     6          129
4  2304342708  1377  chat               DarkRaider   gg     6          129
           match_id  time  type           unit                    key  slot  \
1120643  2306492515   820  chat  ПpOsTo ПyшkиH                      )     6   
1120644  2306492515  2128  chat  **((JOKER))**  this is all i can do      3   
1120645  2306492515  2129  chat  **((JOKER))**                    :D      3   
1120646  2306492515  2141  chat            Stp       англичанини лять     8   
1120647  2306492515  2151  chat            Stp              зиз из...     8   

         player_slot  
1120643          129  
1120644            3  
112064

In [3]:
train_subj_docs = subj_docs[:80]
test_subj_docs = subj_docs[80:100]
train_obj_docs = obj_docs[:80]
test_obj_docs = obj_docs[80:100]
training_docs = train_subj_docs+train_obj_docs
testing_docs = test_subj_docs+test_obj_docs
sentim_analyzer = SentimentAnalyzer()
all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_docs])


In [4]:
unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)
len(unigram_feats)

sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)


In [5]:
training_set = sentim_analyzer.apply_features(training_docs)
test_set = sentim_analyzer.apply_features(testing_docs)


In [6]:
trainer = NaiveBayesClassifier.train
classifier = sentim_analyzer.train(trainer, training_set)
# Training classifier
for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
    print('{0}: {1}'.format(key, value))

Training classifier
Evaluating NaiveBayesClassifier results...
Accuracy: 0.8
F-measure [obj]: 0.8
F-measure [subj]: 0.8
Precision [obj]: 0.8
Precision [subj]: 0.8
Recall [obj]: 0.8
Recall [subj]: 0.8


In [7]:
# 얘는 안되겠다... stfu 이런거 못알아들음ㅠㅠ
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentences = ["VADER is smart, handsome, and funny.", # positive sentence example
"Today sux",     #  negative slang handled
"Today sux!",    #  negative slang with punctuation emphasis handled
"Today SUX!",    #  negative slang with capitalization emphasis
"Today kinda sux! But I'll get by, lol" # mixed sentiment example with slang and constrastive conjunction "but"
]

paragraph = "It was one of the worst movies I've seen, despite good reviews. \
... Unbelievably bad acting!! Poor direction. VERY poor production. \
... The movie was bad. Very bad movie. VERY bad movie. VERY BAD movie. VERY BAD movie!"
from nltk import tokenize


sid = SentimentIntensityAnalyzer()
for sentence in docs:
    print(sentence)
    ss = sid.polarity_scores(sentence)
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k]), end='')
        print()

gg
compound: 0.296, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
gg
compound: 0.296, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
^^
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
thx
compound: 0.3612, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
gg
compound: 0.296, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
brooken
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
lo
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
lol
compound: 0.4215, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
ОК
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
FGJRIW)JGFW
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
ICE FROG PLEASE
compound: 0.3182, 
neg: 0.0, 
neu: 0.465, 
pos: 0.535, 
а ты отличный парень
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
хорошо играешь
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
lol u suck
compound: -0.0258, 
neg: 0.509, 
neu: 0.0, 
pos: 0.491, 
у меня
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
not really gg
compound: -0.2743, 
neg: 0.513, 
neu: 0.487, 
pos: 0.0, 
hilarious
compound: 0.4019, 
neg: 0.0, 
neu: 0.0, 


no
compound: -0.296, 
neg: 1.0, 
neu: 0.0, 
pos: 0.0, 
plz
compound: 0.0772, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
osforg
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
TY
compound: 0.3818, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
dota2 suddenly crashed, idk why
compound: -0.1027, 
neg: 0.259, 
neu: 0.741, 
pos: 0.0, 
rip untouchable
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
awkward
compound: -0.1531, 
neg: 1.0, 
neu: 0.0, 
pos: 0.0, 
1st skill
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
nice
compound: 0.4215, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
stop duelling me pls :(
compound: -0.5859, 
neg: 0.607, 
neu: 0.238, 
pos: 0.155, 
told ya
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
to stop duelling me
compound: -0.296, 
neg: 0.423, 
neu: 0.577, 
pos: 0.0, 
HAHAHA
compound: 0.5574, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
that moves
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
End
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
HAHA
compound: 0.4588, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
Wo

neu: 1.0, 
pos: 0.0, 
XD
compound: 0.5859, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
XDXDXDXXDXD
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
I WILL ASSASSINATE YOUR MMR
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
O_o
compound: -0.128, 
neg: 1.0, 
neu: 0.0, 
pos: 0.0, 

compound: 0.0, 
neg: 0.0, 
neu: 0.0, 
pos: 0.0, 
ass ass in ate?
compound: -0.7906, 
neg: 0.778, 
neu: 0.222, 
pos: 0.0, 
take my win streak away first
compound: 0.5859, 
neg: 0.0, 
neu: 0.568, 
pos: 0.432, 

compound: 0.0, 
neg: 0.0, 
neu: 0.0, 
pos: 0.0, 

compound: 0.0, 
neg: 0.0, 
neu: 0.0, 
pos: 0.0, 

compound: 0.0, 
neg: 0.0, 
neu: 0.0, 
pos: 0.0, 
?
compound: 0.0, 
neg: 0.0, 
neu: 0.0, 
pos: 0.0, 
worth
compound: 0.2263, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
nope...
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 

compound: 0.0, 
neg: 0.0, 
neu: 0.0, 
pos: 0.0, 
hey
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
how you doing?
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
he is lagging
compound: -0.2732,

pos: 0.0, 
but u died
compound: -0.7096, 
neg: 0.831, 
neu: 0.169, 
pos: 0.0, 
but u retard
compound: -0.6808, 
neg: 0.821, 
neu: 0.179, 
pos: 0.0, 
for ur arrows ?
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
need this bane
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
hahahaha
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
phatetic
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
LOL
compound: 0.4215, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
hahahaha
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
nice arrow
compound: 0.4215, 
neg: 0.0, 
neu: 0.263, 
pos: 0.737, 
ty :)
compound: 0.6808, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
u died noob
compound: -0.5859, 
neg: 1.0, 
neu: 0.0, 
pos: 0.0, 
not 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
just committted suicide
compound: -0.6705, 
neg: 0.692, 
neu: 0.308, 
pos: 0.0, 
hahaha
compound: 0.5574, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
but u died
compound: -0.7096, 
neg: 0.831, 
neu: 0.169, 
pos: 0.0, 
lol
compound: 0.4215, 
neg: 0.0, 
neu: 0

neg: 0.0, 
neu: 0.0, 
pos: 0.0, 
О_о
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
кк
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
шторм орчид против кого? 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
чо там про тиму?
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
ну ты 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
и шторм
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
сами
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
подставились
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
на такой тупой мув
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
лайттухевен?
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
лк ласт и фп энигма
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
уиииии
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
ШТОРМ
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
да заткнись
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
ты уже
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
уииии
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
раунд
comp

...

neu: 1.0, 
pos: 0.0, 
lol
compound: 0.4215, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
end
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
gg
compound: 0.296, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
feeder carry
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
?
compound: 0.0, 
neg: 0.0, 
neu: 0.0, 
pos: 0.0, 
noob am
compound: -0.0516, 
neg: 0.545, 
neu: 0.455, 
pos: 0.0, 
noooo
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
Suck
compound: -0.4404, 
neg: 1.0, 
neu: 0.0, 
pos: 0.0, 
just report am
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
Guys]
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
sniper free hit always haha
compound: 0.743, 
neg: 0.0, 
neu: 0.323, 
pos: 0.677, 
PRO AM
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
yej
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
Commended
compound: 0.4404, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
so clean
compound: 0.4576, 
neg: 0.0, 
neu: 0.25, 
pos: 0.75, 
EZ TOP
compound: 0.5106, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
take divine guys
compound: 0.

neu: 1.0, 
pos: 0.0, 
xd
compound: 0.5859, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
XD
compound: 0.5859, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
XD
compound: 0.5859, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
XD
compound: 0.5859, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
How do i out level you guys
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
i had the worst game ever
compound: -0.6249, 
neg: 0.506, 
neu: 0.494, 
pos: 0.0, 
stfu retard
compound: -0.5267, 
neg: 0.773, 
neu: 0.227, 
pos: 0.0, 
invoker
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
xd
compound: 0.5859, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
not really
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
invoker has nothing 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
to do with how shit your levels
compound: -0.5574, 
neg: 0.375, 
neu: 0.625, 
pos: 0.0, 
XD
compound: 0.5859, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
adn farm are
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
really?
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
yup
compound: 0.0, 
neg:

neu: 1.0, 
pos: 0.0, 
fail
compound: -0.5423, 
neg: 1.0, 
neu: 0.0, 
pos: 0.0, 
fuck
compound: -0.5423, 
neg: 1.0, 
neu: 0.0, 
pos: 0.0, 
me 2 bro
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
hahahahaha
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
./.
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
throw?
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
ggwp
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
Wp haha
compound: 0.6124, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
INVOKER NOOB
compound: -0.0516, 
neg: 0.545, 
neu: 0.455, 
pos: 0.0, 
buhat na buhat tong -putang inang tong mga to
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
MALEV ONLY
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
napaka tatnaga
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
damn
compound: -0.4019, 
neg: 1.0, 
neu: 0.0, 
pos: 0.0, 
BOBO KA GAGO
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
mas bobo ka tanga
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
MALEV LNG DI MO PA MAPINDOT
compound: 0.0, 
n

pos: 0.0, 
but u counter zeus
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
Gg
compound: 0.296, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
gg tards
compound: 0.296, 
neg: 0.0, 
neu: 0.312, 
pos: 0.688, 
try more
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
report this zeus
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
Please
compound: 0.3182, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
toxic player
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
to counter 1 hero
compound: 0.5574, 
neg: 0.0, 
neu: 0.357, 
pos: 0.643, 
6-15 mid, got to be fun :)
compound: 0.743, 
neg: 0.0, 
neu: 0.442, 
pos: 0.558, 
rofl
compound: 0.5719, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
3 hero
compound: 0.5574, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
vs me
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
ez zeus
compound: 0.3612, 
neg: 0.0, 
neu: 0.286, 
pos: 0.714, 
;)
compound: 0.2263, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
boje
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
ebal prosto
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0

pos: 0.0, 
have an item
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
see
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
how 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
he 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
move
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
gg
compound: 0.296, 
neg: 0.0, 
neu: 0.0, 
pos: 1.0, 
im just 2k1 solo mmr
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
no one cares anyway
compound: 0.2023, 
neg: 0.306, 
neu: 0.278, 
pos: 0.417, 
you have better oickl
compound: 0.4404, 
neg: 0.0, 
neu: 0.508, 
pos: 0.492, 
i dont even mind
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
if you 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
pick
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
6k or 7k
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
thats enough
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
thats enough
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
you shit
compound: -0.5574, 
neg: 0.783, 
neu: 0.217, 
pos: 0.0, 
go back 

0
1
2


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278


1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062


3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701


5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342


6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981


8423
8424
8425
8426
8427
8428
8429
8430
8431
8432
8433
8434
8435
8436
8437
8438
8439
8440
8441
8442
8443
8444
8445
8446
8447
8448
8449
8450
8451
8452
8453
8454
8455
8456
8457
8458
8459
8460
8461
8462
8463
8464
8465
8466
8467
8468
8469
8470
8471
8472
8473
8474
8475
8476
8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612
8613
8614
8615
8616
8617
8618
8619
8620
8621
8622


10052
10053
10054
10055
10056
10057
10058
10059
10060
10061
10062
10063
10064
10065
10066
10067
10068
10069
10070
10071
10072
10073
10074
10075
10076
10077
10078
10079
10080
10081
10082
10083
10084
10085
10086
10087
10088
10089
10090
10091
10092
10093
10094
10095
10096
10097
10098
10099
10100
10101
10102
10103
10104
10105
10106
10107
10108
10109
10110
10111
10112
10113
10114
10115
10116
10117
10118
10119
10120
10121
10122
10123
10124
10125
10126
10127
10128
10129
10130
10131
10132
10133
10134
10135
10136
10137
10138
10139
10140
10141
10142
10143
10144
10145
10146
10147
10148
10149
10150
10151
10152
10153
10154
10155
10156
10157
10158
10159
10160
10161
10162
10163
10164
10165
10166
10167
10168
10169
10170
10171
10172
10173
10174
10175
10176
10177
10178
10179
10180
10181
10182
10183
10184
10185
10186
10187
10188
10189
10190
10191
10192
10193
10194
10195
10196
10197
10198
10199
10200
10201
10202
10203
10204
10205
10206
10207
10208
10209
10210
10211
10212
10213
10214
10215
10216
10217
1021